In [132]:

def parseNumber(line, start=0):
    if line[start] == '[':
        return parsePair(line, start)
    else:
        return int(line[start]), start + 1

def parsePair(line, start):
    first, middle = parseNumber(line, start + 1)
    second, end = parseNumber(line, middle + 1)
    
    while end < len(line) and line[end] == ']':
        end += 1
    return [first, second], end

def getInput(inputFile):
    with open(inputFile, "r") as file:
        line = file.read().split('\n')
        return map(lambda x: parseNumber(x)[0], filter(lambda x: len(x) != 0, line))

In [133]:
def getValueAtPath(number, path):
    current = number
    for element in path:
        if isinstance(current, int):
            return None
        current = current[element]
    return current

def allValues(number):
    path = []

    # Find the first scalar in the number
    while not isinstance(getValueAtPath(number, path), (int, type(None))):
        path.append(0)

    # Iterate though the numbers
    while path != None:
        yield getValueAtPath(number, path), path
        path = getPathOfRightNumber(number, path)

def checkNumberForNestedPairs(number, depth, path=None):
    if path == None:
        path = []
        result = checkNumberForNestedPairs(number, depth, path)
        return result, path

    if isinstance(number, int):
        return False

    if depth == 0:
        return True

    if isinstance(number, int):
        return False

    path.append(0)
    if checkNumberForNestedPairs(number[0], depth - 1, path):
        return True
    
    path.pop()
    path.append(1)
    if checkNumberForNestedPairs(number[1], depth - 1, path):
        return True
    path.pop()

    return False

def checkNumberForLargeScalars(number, maximum):
    for scalar, path in allValues(number):
        if scalar > maximum:
            return True, path
    return False, None

def getPathOfLeftNumber(number, path):
    result = path[:]

    # Unwind the path until we find a pair we are on the right hand side of
    while len(result) > 0 and result[-1] != 1:
        result.pop()

    if len(result) == 0:
        return None

    # Switch the end of the path to the left hand side
    result[-1] = 0

    # Keep taking the right hand side until we find a scalar value
    while not isinstance(getValueAtPath(number, result), (int, type(None))):
        result.append(1)

    return result
        
def getPathOfRightNumber(number, path):
    result = path[:]

    # Unwind the path until we find a pair we are on the left hand side of
    while len(result) > 0 and result[-1] != 0:
        result.pop()

    if len(result) == 0:
        return None

    # Switch the end of the path to the right hand side
    result[-1] = 1

    # Keep taking the left hand side until we find a scalar value
    while not isinstance(getValueAtPath(number, result), (int, type(None))):
        result.append(0)

    return result

def explodePair(number, path):
    pair = getValueAtPath(number, path)
    left = getPathOfLeftNumber(number, path)
    right = getPathOfRightNumber(number, path)

    if left != None:
        getValueAtPath(number, left[:-1])[left[-1]] += pair[0]

    if right != None:
        getValueAtPath(number, right[:-1])[right[-1]] += pair[1]
    
    getValueAtPath(number, path[:-1])[path[-1]] = 0

def splitNumber(number, path):
    value = getValueAtPath(number, path)
    pair = [value // 2, value - value // 2]
    getValueAtPath(number, path[:-1])[path[-1]] = pair

def reduceNumber(number):
    while True:
        shouldExplode, path = checkNumberForNestedPairs(number, 4)
        if shouldExplode:
            explodePair(number, path)
            continue
        shouldSplit, path = checkNumberForLargeScalars(number, 9)
        if shouldSplit:
            splitNumber(number, path)
            continue
        break

def addNumbers(a, b):
    result = [a, b]
    reduceNumber(result)
    return result

def findMagnitude(number):
    if isinstance(number, int):
        return number
    else:
        return 3*findMagnitude(number[0]) + 2*findMagnitude(number[1])

In [134]:
for value in getInput("example-1.txt"):
    success, path = checkNumberForNestedPairs(value, 4)
    if success:
        old = str(value)
        explodePair(value, path)
        print(f'{old: <50} -> {value}')

[[[[[9, 8], 1], 2], 3], 4]                         -> [[[[0, 9], 2], 3], 4]
[7, [6, [5, [4, [3, 2]]]]]                         -> [7, [6, [5, [7, 0]]]]
[[6, [5, [4, [3, 2]]]], 1]                         -> [[6, [5, [7, 0]]], 3]
[[3, [2, [1, [7, 3]]]], [6, [5, [4, [3, 2]]]]]     -> [[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
[[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]          -> [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]


In [135]:
import functools

def partOne(inputFile):
    finalSum = functools.reduce(addNumbers, getInput(inputFile))
    return findMagnitude(finalSum)

In [136]:
print(partOne("example-2.txt"))
print(partOne("input.txt"))

4140
4202


In [137]:
def getAllCombinations(inputFile):
    # We end up mutating the numbers when we add them, so I have coded around this here
    # defiantly not ideal but good enough for now :)
    numbers = list(getInput(inputFile))
    for i in range(len(numbers)):
        for j in range(len(numbers)):
            if i != j:
                a = list(getInput(inputFile))[i]
                b = list(getInput(inputFile))[j]
                yield findMagnitude(addNumbers(a, b))

def partTwo(inputFile):
    return max(getAllCombinations(inputFile))



In [139]:
print(partTwo("example-2.txt"))
print(partTwo("input.txt"))

3993
4779
